In [10]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [79]:
base_url = 'https://www.kotsovolos.gr'
#url = 'https://www.kotsovolos.gr/site/sound-vision/televisions/led-lcd?v=0' #13 products
#url = 'https://www.kotsovolos.gr/site/sound-vision/televisions/led-lcd?v=0&flt=100' #1st tv page
#url = 'https://www.kotsovolos.gr/site/sound-vision/televisions/led-lcd?v=0&flt=100&page=1' #2nd tv page
#url = 'https://www.kotsovolos.gr/site/sound-vision/televisions/led-lcd?v=0&flt=100&page=2' #3rd tv page
#url = 'https://www.kotsovolos.gr/site/household-appliances/fridges/fridge-freezers?v=0&flt=100' #Fridges
#url = 'https://www.kotsovolos.gr/site/search.jsp?v=0&searchfor=BBBK1BYajVlbmg=&flt=1000' #Search ΨΥΓΕΙΑ
#url = 'https://www.kotsovolos.gr/site/search.jsp?v=0&searchfor=BBBYkdGd2RHOXc=&flt=1000' #Search laptop
#url = 'https://www.kotsovolos.gr/site/search.jsp?v=0&searchfor=BBBN09mMzRlM24=&flt=10000' #Search Μηχανή
#url = 'https://www.kotsovolos.gr/site/search.jsp?v=0&searchfor=BBBNnVudDUvVHY=&flt=10000' #Search κινητό
#url = 'https://www.kotsovolos.gr/site/search.jsp?v=0&searchfor=BBBNGU3bDgrLzE0ZkU9&flt=10000' # Search αξεσουάρ
#url = 'https://www.kotsovolos.gr/site/search.jsp?v=0&searchfor=BBBOE92MTdmVGU4ZW52&flt=1000' # Search πλυντήριο
url = 'https://www.kotsovolos.gr/site/search.jsp?v=0&searchfor=BBBNnVIczVmSGg=&flt=1000' # Search κάμερα

In [80]:
#Scraping that does not support javascript generated content
response = requests.get(url)
print(response)
# 200 means it got through
soup = BeautifulSoup(response.text, 'html.parser')
mydivs = soup.findAll("a")

# This does not work, all products have #yreviews, even of they don't have reviews
products = list(filter(lambda x: '#yreviews' in x['href'], mydivs))

print(f'Products: {len(products)}')

<Response [200]>
Products: 461


In [81]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [66]:
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
#In case of error upgrade your chrome browser


Checking for win32 chromedriver:74.0.3729.6 in cache
Driver found in C:\Users\s.lagousis.POBUCA\.wdm\chromedriver\74.0.3729.6\win32/chromedriver.exe


In [82]:
# Use selenium to get list of products to avoid products without reviews
driver.get(url)
time.sleep(60)
all_products_html = driver.page_source
all_products_soup = BeautifulSoup(all_products_html, 'html.parser')

In [84]:
products = []
all_products = all_products_soup.findAll("div", class_="productdata")
for product in all_products:
    if len(product.findAll('span', class_='yotpo-stars'))>0:
        products.append(product.findAll('a')[0]['href'])
print(f'Products with reviews: {len(products)}')

Products with reviews: 197


In [85]:
def GetRewiewsOfProduct(product_soup, reviews_list):
    reviews = product_soup.findAll("div", class_="yotpo-review yotpo-regular-box")
    #print(f'Reviews: {len(reviews)}')

    for review in reviews:
        stars = len(review.findAll('span', class_='yotpo-icon yotpo-icon-star rating-star pull-left'))
        #print(f'Stars: {stars}')
        review_title = review.findAll('div', class_='content-title yotpo-font-bold')[0].text
        #print(review_title)
        review_text = review.findAll('div', class_='content-review')[0].text
        #print(review_text)
        #new_review = {'stars': stars, 'title': review_title, 'review': review_text}
        new_review = {'stars': stars, 'text': review_text}
        reviews_list.append(new_review)

In [86]:
from random import randint

i = 0
reviews_list = []

for product in products:
    i+=1
    printProgressBar(i, len(products), prefix=f'Reviews so far:{len(reviews_list)}', length=60)
        
    product_url = base_url + product #['href']
    
    # This does not support content created by javascript 
    #product_response = requests.get(url)
    #product_soup = BeautifulSoup(product_response.text, 'html.parser')
    
    # This supports content created by javascript 
    driver.get(product_url)
    time.sleep(randint(3, 5))
    product_html = driver.page_source
    product_soup = BeautifulSoup(product_html, 'html.parser')
    GetRewiewsOfProduct(product_soup, reviews_list)
        
    #break

Reviews so far:684 |████████████████████████████████████████████████████████████| 100.0% 


In [87]:
len(reviews_list), reviews_list[:10]

(686,
 [{'stars': 5, 'text': ' extremely easy to set up and use. Great product! '},
  {'stars': 5,
   'text': ' Πολύ ευχαριστημένος! , Ασύρματη λειτουργία καλή εικόνα πρωί-βράδυ, zoom, δυνατότητα περιστροφής, ομιλίας μέσα από την κάμερα, παρακολούθησης live από το κινητό-tablet,pc όποτε θέλεις, Μotion detection πολύ καλό όπως και ήχου detection επίσης καλό. και Καταγράφει 24 ώρες το εικοσιτετράωρο, ενημερώνει με app κινητό-tablet, και στέλνει 3 φωτογραφίες ...Διαβάστε Περισσότεραστο mail σου.(Μέχρι 60 mail το μήνα με δυνατότητα επέκτασης του αριθμού των mail). Διαβάστε Λιγότερα'},
  {'stars': 4,
   'text': ' το χρησιμοποιώ 1 περίπου μήνα και προς το παρόν μόνον ασύρματα.  ΘΕΤΙΚΑ:  - Εύκολη σύνδεση - Πολύ καλό για εποπτεία του χώρου (καταγραφή δεν έχω ακόμη δοκιμάσει) - Ευαίσθητοι αισθητήρες  - Άμεση ειδοποίηση για συμβάντα (το ενεργοποίησα δοκιμαστικά για λίγο), αλλά όπως διάβασα με κάποιους περιορισμούς στο πλήθος των ειδοποιήσεων. ...Διαβάστε Περισσότερα- Έλεγχος με πολλούς τρόπους (

In [89]:
#Save to file
import unicodecsv as csv

with open('reviews_kotsovolos.csv', 'wb') as csvfile: 
    keys = reviews_list[0].keys()
    dict_writer = csv.DictWriter(csvfile, keys)
    dict_writer.writeheader()
    dict_writer.writerows(reviews_list)